In [ ]:
import os

import bids

from nipype.interfaces.utility import IdentityInterface
from nipype.interfaces.io import BIDSDataGrabber
from nipype.pipeline.engine import Node, Workflow
from nipype import config

# config.enable_debug_mode()
bids.config.set_option('extension_initial_dot', True)


## Load custom derivatives inputs with `BIDSDataGrabber` 

In [ ]:
bids_dir = os.path.abspath("/home/localadmin/Desktop/hcp-retest-d2")
custom_derivatives_dir = os.path.join(bids_dir, 'derivatives', 'cmp')
subject = "103818"
session = "01"

wf = Workflow(
    base_dir=os.path.join(bids_dir, 'code'),
    name='bids_grabber_workflow'
)
wf.config['execution'] = {
    'use_relative_paths': 'True'
}


bg = Node(
    interface=BIDSDataGrabber(),
    name="bg",
    base_dir=os.path.join(wf.base_dir),
)
bg.inputs.base_dir = bids_dir
bg.inputs.extra_derivatives = [custom_derivatives_dir]
bg.inputs.subject = subject
bg.inputs.session = session
bg.inputs.output_query = {
    "T1w": {
        "datatype": "anat",
        "suffix": "T1w",
        "extension": ["nii", ".nii.gz"],
    },
    "dwi": {
        "datatype": "dwi",
        "suffix": "dwi",
        "extension": ["nii", ".nii.gz"],
    },
    "bold": {
        "datatype": "func",
        "suffix": "bold",
        "extension": ["nii", ".nii.gz"],
    },
    "roi_volumes": {
        "suffix": "dseg", 
        "extension": ".nii.gz",
        "atlas": "L2018"
    },
     "wm_mask": {
        "suffix": "dseg", 
        "extension": ".nii.gz",
        "label": "WM"
    },
    "gm_mask": {
        "suffix": "dseg", 
        "extension": ".nii.gz",
        "label": "GM"
    },
    "csf_mask": {
        "suffix": "dseg", 
        "extension": ".nii.gz",
        "label": "CSF"
    },
}
bg.inputs.raise_on_empty = True

ii = Node(
    interface=IdentityInterface(
        fields=['T1w', 'dwi', 'roi_volumes', 'wm_mask', 'gm_mask', 'csf_mask'],
        mandatory_inputs=True
    ),
    name='ii'
)
    
wf.connect(
     [
         (bg, ii,  [('T1w', 'T1w'),
                    ('dwi', 'dwi'),
                    ('bold', 'bold'),
                    ('roi_volumes', 'roi_volumes'),
                    ('wm_mask', 'wm_mask'),
                    ('gm_mask', 'gm_mask'),
                    ('csf_mask', 'csf_mask')])
     ]
)

wf.run()

In [ ]:
wf.get_node('bg').result.outputs


In [ ]:
import gzip, pickle

node='bg'

with gzip.open(
    os.path.join(bids_dir, 'code', 'bids_grabber_workflow', node, f'result_{node}.pklz'), 'rb'
) as fp:
    content = fp.read()
    outputs = pickle.loads(content)

    print(outputs.inputs)
    print(outputs.outputs)

In [ ]:
import nipype.pipeline.engine.utils as nipype_utils
outputs2 = nipype_utils.load_resultfile(
    os.path.join(bids_dir, 'code', 'bids_grabber_workflow', node, f'result_{node}.pklz'),
    resolve=True
)
print(outputs2.inputs == outputs.inputs)
print(outputs2.outputs)

### How to filter scan in orig space with  `pybids`

In [ ]:
from bids import BIDSLayout

layout = BIDSLayout(bids_dir)
layout.to_df()

In [ ]:
layout.add_derivatives(os.path.join(bids_dir, "derivatives", "cmp"))
layout.to_df()

In [ ]:
files = layout.get(
    suffix="dseg", 
    extension=".nii.gz",
    atlas="L2018"
)
files = [
    file for file in files if "space" not in file.path
]
files

In [ ]:
files = layout.get(
    suffix="dseg", 
    extension=".nii.gz",
    label="WM"
)
files = [
    file for file in files if "space" not in file.path
]
files

## Traits/traitsui model for describing each custom input 

In [1]:
from traits.api import (HasTraits, Directory, Str, Instance)
from traitsui.api import Item, View, VGroup


class CustomBIDSFile(HasTraits):
    
    suffix = Str
    acquisition = Str
    resolution = Str
    extension = Str
    atlas = Str
    label = Str
    desc = Str
    
    traits_view = View
    
    def __init__(
        self,
        p_suffix="",
        p_extension="",
        p_acquisition="",
        p_atlas="",
        p_resolution="",
        p_label="",
        p_desc=""
    ):        
        self.suffix = p_suffix
        self.extension = p_extension
        self.acquisition = p_acquisition
        self.atlas = p_atlas
        self.resolution = p_resolution
        self.label = p_label
        self.desc = p_desc
        
    def __str__(self):
        msg = "{"
        msg += f' "suffix": {self.suffix}, '
        msg += f' "extension": {self.extension}, '
        msg += f' "acquisition": {self.acquisition}, '
        msg += f' "atlas": {self.atlas}, '
        msg += f' "resolution": {self.resolution}, '
        msg += f' "label": {self.label}, '
        msg += f' "desc": {self.desc}'
        msg += "}"
        return msg

    
    
class CustomParcellationBIDSFile(CustomBIDSFile):
    
    traits_view = View(
        VGroup(
            Item('suffix', style='readonly', label='suffix'),
            Item('atlas', label='atlas'),
            Item('resolution', label='res'),
        )
    )

    def __init__(self):
        super().__init__(p_suffix="dseg", p_atlas="L2018")

    
class CustomWMMaskBIDSFile(CustomBIDSFile):
    
    traits_view = View(
        VGroup(
            Item('suffix', style='readonly', label='suffix'),
            Item('label', style='readonly', label='label'),
            Item('desc', label='desc'),
        )
    )
    
    def __init__(self):
        super().__init__(p_suffix="dseg", p_label="WM")

    
class CustomGMMaskBIDSFile(CustomBIDSFile):
    
    traits_view = View(
        VGroup(
            Item('suffix', style='readonly', label='suffix'),
            Item('label', style='readonly', label='label'),
            Item('desc', label='desc'),
        )
    )

    def __init__(self):
        super().__init__(p_suffix="dseg", p_label="GM")


class CustomCSFMaskBIDSFile(CustomBIDSFile):

    traits_view = View(
        VGroup(
            Item('suffix', style='readonly', label='suffix'),
            Item('label', style='readonly', label='label'),
            Item('desc', label='desc'),
        )
    )

    def __init__(self):
        super().__init__(p_suffix="dseg", p_label="CSF")


class MyCustomInstanceInputs(HasTraits):
    
    custom_anatomical_derivatives_dir = Directory
    custom_parcellation = Instance(CustomParcellationBIDSFile, ())
    custom_wm_mask = Instance(CustomWMMaskBIDSFile, ())
    custom_gm_mask = Instance(CustomGMMaskBIDSFile, ())
    custom_csf_mask = Instance(CustomCSFMaskBIDSFile, ())
    
    view = View(

        Item('custom_anatomical_derivatives_dir'),
        Item('custom_parcellation', style='custom'),
        Item('_'),
        Item('custom_wm_mask', style='custom', label='Custom WM mask'),
        Item('_'),
        Item('custom_gm_mask', style='custom', label='Custom GM mask'),
        Item('_'),
        Item('custom_csf_mask', style='custom', label='Custom CSF mask'),

        # View options
        resizable = True,
        width=500,
        height=400,
        title = 'Custom anatomical pipeline inputs',
    )
    

my_custom_instance_inputs = MyCustomInstanceInputs()
my_custom_instance_inputs.configure_traits()

True

In [2]:
my_custom_instance_inputs.custom_parcellation.traits()

{'suffix': <traits.ctrait.CTrait at 0x7f8687cefb50>,
 'acquisition': <traits.ctrait.CTrait at 0x7f868c236940>,
 'resolution': <traits.ctrait.CTrait at 0x7f8687cf81b0>,
 'extension': <traits.ctrait.CTrait at 0x7f868c236c00>,
 'atlas': <traits.ctrait.CTrait at 0x7f8687c99940>,
 'label': <traits.ctrait.CTrait at 0x7f868c236ec0>,
 'desc': <traits.ctrait.CTrait at 0x7f868c237050>,
 'trait_added': <traits.ctrait.CTrait at 0x7f86aca83310>,
 'trait_modified': <traits.ctrait.CTrait at 0x7f86aca83520>}

## Save the object traits into a json file with the help of `configparser`

In [3]:
import configparser
from inspect import ismethod

config = configparser.RawConfigParser()

# Add stage section and corresponding parameters
if hasattr(my_custom_instance_inputs, "traits") and ismethod(getattr(my_custom_instance_inputs, "traits")):
    config.add_section('CustomBIDSInputs')
    
    keys = [
        prop for prop in list(my_custom_instance_inputs.traits().keys()) if "trait" not in prop
    ]  # possibly dangerous..?
    for key in keys:
        
        print(key)
        keyval = getattr(my_custom_instance_inputs, key)

        if key in ['custom_parcellation', 'custom_wm_mask', 'custom_gm_mask', 'custom_csf_mask']:
            
            sub_keys = [
                prop for prop in list(keyval.traits().keys()) if "trait" not in prop
            ]  # possibly dangerous..?
            for sub_key in sub_keys:
                
                print(sub_key)
                config.set(
                    'CustomBIDSInputs',
                    key + "." + sub_key,
                    getattr(keyval, sub_key)
                )
        else:
            config.set(
                'CustomBIDSInputs',
                key,
                keyval
            )
            


custom_anatomical_derivatives_dir
custom_parcellation
suffix
acquisition
resolution
extension
atlas
label
desc
custom_wm_mask
suffix
acquisition
resolution
extension
atlas
label
desc
custom_gm_mask
suffix
acquisition
resolution
extension
atlas
label
desc
custom_csf_mask
suffix
acquisition
resolution
extension
atlas
label
desc


In [4]:
print(config)

In [5]:
import json
from collections.abc import Iterable

config_json = {}
debug = True

for section in config.sections():
    config_json[section] = {}
    for name, value in config.items(section):
        # Keep only parameters that are used by the diffusion stage
        # of the diffusion pipeline. This simplifies the reading of
        # its configuration file
        
        if value:

            is_iterable = False

            try:
                value = eval(value)
                if debug:
                    print_warning(f"  .. DEBUG: String {value} evaluated")

            except Exception:
                if debug:
                    print(
                        f"  .. EXCEPTION: String {value} COULD NOT BE evaluated"
                    )
                pass

            if isinstance(value, dict):
                if debug:
                    print(
                        f"  .. DEBUG: Processing {section} / {name} / {value} as dict"
                    )
                config_json[section][name] = value
                is_iterable = True
            elif isinstance(value, list):
                if debug:
                    print(
                        f"  .. DEBUG: Processing {section} / {name} / {value} as list"
                    )
                config_json[section][name] = value
                is_iterable = True
            elif isinstance(value, Iterable) and not isinstance(value, str):
                if debug:
                    print(
                        f"  .. DEBUG: Processing {section} / {name} / {value} as iterable"
                    )
                config_json[section][name] = [x for x in value if x]
                is_iterable = True
            elif isinstance(value, bool):
                if debug:
                    print(
                        f"  .. DEBUG: Processing {section} / {name} / {value} as boolean"
                    )
                config_json[section][name] = [value]
            elif value and not isinstance(value, str):
                if debug:
                    print(
                        f"  .. DEBUG: Processing {section} / {name} / {value} as not a string"
                    )
                config_json[section][name] = [value]
            elif value and isinstance(value, str):
                value = value.strip()
                if value.isnumeric():
                    if debug:
                        print(
                            f"  .. DEBUG: Processing {section} / {name} / {value} as number"
                        )
                    value = float(value)
                    if value.is_integer():
                        value = int(value)
                    config_json[section][name] = [value]
                else:
                    if debug:
                        print(
                            f"  .. DEBUG: Processing {section} / {name} / {value} as string"
                        )
                    config_json[section][name] = [value]
            else:
                if debug:
                    print(f"  .. DEBUG : Type: {type(value)} / value : {value}")
                config_json[section][name] = ""

            if not is_iterable:
                if len(config_json[section][name]) == 1:
                    config_json[section][name] = config_json[section][name][0]
                elif len(config_json[section][name]) == 0:
                    config_json[section][name] = ""

            if config_json[section][name] == "":
                del config_json[section][name]

print(f"  .. DEBUG: {config_json}")

with open('custom_config.json', "w") as outfile:
    json.dump(config_json, outfile, indent=4)

  .. EXCEPTION: String dseg COULD NOT BE evaluated
  .. DEBUG: Processing CustomBIDSInputs / custom_parcellation.suffix / dseg as string
  .. EXCEPTION: String L2018 COULD NOT BE evaluated
  .. DEBUG: Processing CustomBIDSInputs / custom_parcellation.atlas / L2018 as string
  .. EXCEPTION: String dseg COULD NOT BE evaluated
  .. DEBUG: Processing CustomBIDSInputs / custom_wm_mask.suffix / dseg as string
  .. EXCEPTION: String WM COULD NOT BE evaluated
  .. DEBUG: Processing CustomBIDSInputs / custom_wm_mask.label / WM as string
  .. EXCEPTION: String dseg COULD NOT BE evaluated
  .. DEBUG: Processing CustomBIDSInputs / custom_gm_mask.suffix / dseg as string
  .. EXCEPTION: String GM COULD NOT BE evaluated
  .. DEBUG: Processing CustomBIDSInputs / custom_gm_mask.label / GM as string
  .. EXCEPTION: String dseg COULD NOT BE evaluated
  .. DEBUG: Processing CustomBIDSInputs / custom_csf_mask.suffix / dseg as string
  .. EXCEPTION: String CSF COULD NOT BE evaluated
  .. DEBUG: Processing C

## Initialize a `MyCustomInstanceInputs` object with the config json file

In [6]:
with open('custom_config.json', "r") as f:
    config = json.load(f)
    
new_my_custom_instance_inputs = MyCustomInstanceInputs()
    

keys = [
    prop for prop in list(new_my_custom_instance_inputs.traits().keys()) if "trait" not in prop
]  # possibly dangerous..?
for key in keys:
    
    print(key)
    keyval = getattr(new_my_custom_instance_inputs, key)
    
    if key in ['custom_parcellation', 'custom_wm_mask', 'custom_gm_mask', 'custom_csf_mask']:

        sub_keys = [
            prop for prop in list(keyval.traits().keys()) if "trait" not in prop
        ]  # possibly dangerous..?
        for sub_key in sub_keys:

            print(sub_key)
            tmp_key = key + "." + sub_key
            if tmp_key in config['CustomBIDSInputs'].keys():
                conf_value = config['CustomBIDSInputs'][tmp_key]
                print(conf_value)
                try:
                    # Convert parameter to proper expected type
                    if isinstance(getattr(keyval, sub_key), tuple):
                        conf_value = tuple(conf_value)
                    elif isinstance(getattr(keyval, sub_key), bool):
                        conf_value = bool(conf_value)
                    elif isinstance(getattr(keyval, sub_key), list):
                        conf_value = list(conf_value)
                    elif isinstance(getattr(keyval, sub_key), dict):
                        conf_value = dict(conf_value)
                    elif isinstance(getattr(keyval, sub_key), int):
                        conf_value = int(float(conf_value))
                    elif isinstance(getattr(keyval, sub_key), float):
                        conf_value = float(conf_value)
                    setattr(keyval, sub_key, conf_value)
                    if debug:
                        print(
                            f" .. DEBUG: Set {sub_key} to {conf_value}"
                        )
                except Exception as e:
                    if debug:
                        print(
                            "  .. EXCEPTION raised while setting "
                            + f"{sub_key} to {conf_value}"
                        )
                        print(f"    {e}")
                pass
    else:
         if key in config['CustomBIDSInputs'].keys():
                conf_value = config['CustomBIDSInputs'][key]
                try:
                    # Convert parameter to proper expected type
                    if isinstance(keyval, tuple):
                        conf_value = tuple(conf_value)
                    elif isinstance(keyval, bool):
                        conf_value = bool(conf_value)
                    elif isinstance(keyval, list):
                        conf_value = list(conf_value)
                    elif isinstance(keyval, dict):
                        conf_value = dict(conf_value)
                    elif isinstance(keyval, int):
                        conf_value = int(float(conf_value))
                    elif isinstance(keyval, float):
                        conf_value = float(conf_value)
                    setattr(new_my_custom_instance_inputs, key, conf_value)
                    if debug:
                        print(
                            f" .. DEBUG: Set {key} to {conf_value}"
                        )
                except Exception as e:
                    if debug:
                        print_warning(
                            "  .. EXCEPTION raised while setting "
                            + f"{key} to {conf_value}"
                        )
                        print_error(f"   {e}")
                    pass


custom_anatomical_derivatives_dir
custom_parcellation
suffix
dseg
 .. DEBUG: Set suffix to dseg
acquisition
resolution
extension
atlas
L2018
 .. DEBUG: Set atlas to L2018
label
desc
custom_wm_mask
suffix
dseg
 .. DEBUG: Set suffix to dseg
acquisition
resolution
extension
atlas
label
WM
 .. DEBUG: Set label to WM
desc
custom_gm_mask
suffix
dseg
 .. DEBUG: Set suffix to dseg
acquisition
resolution
extension
atlas
label
GM
 .. DEBUG: Set label to GM
desc
custom_csf_mask
suffix
dseg
 .. DEBUG: Set suffix to dseg
acquisition
resolution
extension
atlas
label
CSF
 .. DEBUG: Set label to CSF
desc


In [9]:
print(f'{new_my_custom_instance_inputs.custom_parcellation}')

{ "suffix": dseg,  "extension": ,  "acquisition": ,  "atlas": L2018,  "resolution": ,  "label": ,  "desc": }
